# Person Re-Identification with Body Part-Based Features and PSFP

This Jupyter Notebook implements a Person Re-Identification (Re-ID) system using a **Body Part-Based Re-Identification (BPBreID)** model with **Progressive Soft Filter Pruning (PSFP)**. The demo matches a query image to a set of gallery images using body part-based features.

## Prerequisites
- Python 3.8 or higher
- Dependencies: `torch`, `torchvision`, `numpy`, `matplotlib`, `tqdm`, `pillow`, `opencv-python`, `gdown`, `torchreid`
- Images in `data/query/` (e.g., `query_image.jpg`) and `data/gallery/` (e.g., `gallery1.jpg`, `gallery2.jpg`, `gallery3.jpg`)

## Setup Instructions
1. Clone the repository: `git clone https://github.com/<your-username>/Person-ReID-PSFP.git`
2. Create and activate a virtual environment:
   - Windows: `python -m venv reid_env; .\reid_env\Scripts\activate`
   - Linux/macOS: `python -m venv reid_env; source reid_env/bin/activate`
3. Run the cell below to install dependencies (last updated: 06:26 PM IST, Wednesday, July 09, 2025).
4. Place images in `data/query/` and `data/gallery/`.
5. Execute the cells in order.

## Notes
- The demo uses a simulated pretrained model (ResNet50 backbone).
- Training functionality is a placeholder; contact the repository owner for a full implementation.
- Report issues at the GitHub repository.

In [2]:
# Cell 1: Install and verify dependencies
import sys
import subprocess

def install_packages():
    packages = ['torch', 'torchvision', 'numpy', 'matplotlib', 'tqdm', 'pillow', 'opencv-python', 'gdown']
    for pkg in packages:
        try:
            __import__(pkg)
        except ImportError:
            print(f'Installing {pkg}...')
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
    try:
        import torchreid
    except ImportError:
        print('Installing torchreid...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'git+https://github.com/KaiyangZhou/deep-person-reid.git'])

install_packages()

# Verify dependencies
try:
    import torch, torchvision, numpy, matplotlib, tqdm, PIL, torchreid, cv2, gdown
    print('All dependencies installed successfully!')
except ImportError as e:
    print(f'Error: {e}. Please run this cell again or install the missing package manually with `pip install <package>`.')

Installing pillow...
Installing opencv-python...
Installing torchreid...


CalledProcessError: Command '['c:\\Users\\sam\\AppData\\Local\\Programs\\Python\\Python310\\python.exe', '-m', 'pip', 'install', 'git+https://github.com/KaiyangZhou/deep-person-reid.git']' returned non-zero exit status 1.

## Model Definitions

This cell defines the `BPBreID` and `BPBreID_PSFP` models, along with supporting classes for body part-based feature extraction and re-identification. The `BPBreID` model uses a ResNet50 backbone and extracts features from body parts, while `BPBreID_PSFP` wraps it with a placeholder for PSFP.

In [3]:
# Cell 2: Model definitions
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image
import os
from torchvision import transforms

# Supporting classes
class GlobalAveragePoolingHead(nn.Module):
    def __init__(self, in_channels):
        super(GlobalAveragePoolingHead, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
    
    def forward(self, x, mask=None):
        if mask is not None:
            x = x * mask
        return self.pool(x)

class PixelToPartClassifier(nn.Module):
    def __init__(self, in_channels, parts_num):
        super(PixelToPartClassifier, self).__init__()
        self.conv = nn.Conv2d(in_channels, parts_num + 1, kernel_size=1)
    
    def forward(self, x):
        return self.conv(x)

class BNClassifier(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(BNClassifier, self).__init__()
        self.bn = nn.BatchNorm1d(in_channels)
        self.fc = nn.Linear(in_channels, num_classes)
    
    def forward(self, x):
        x = self.bn(x)
        return x, self.fc(x)

class BeforePoolingDimReduceLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BeforePoolingDimReduceLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class AfterPoolingDimReduceLayer(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.0):
        super(AfterPoolingDimReduceLayer, self).__init__()
        self.fc = nn.Linear(in_channels, out_channels)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
    
    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dropout(x)
        return x

def init_part_attention_pooling_head(normalization, pooling, dim_reduce_output):
    return GlobalAveragePoolingHead(dim_reduce_output)

# BPBreID Model
class BPBreID(nn.Module):
    def __init__(self, num_classes, pretrained=True, loss='softmax', model_cfg=None):
        super(BPBreID, self).__init__()
        self.num_classes = num_classes
        self.parts_num = model_cfg.get('parts_num', 6) if model_cfg else 6
        self.shared_parts_id_classifier = model_cfg.get('shared_parts_id_classifier', True) if model_cfg else True
        self.dim_reduce_output = model_cfg.get('dim_reduce_output', 512) if model_cfg else 512
        
        # Use ResNet50 as backbone
        self.backbone = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=pretrained)
        self.backbone.fc = nn.Identity()  # Remove final FC layer
        
        # Body part-based components
        self.pixel_to_part_classifier = PixelToPartClassifier(2048, self.parts_num)
        self.before_pooling_dim_reducer = BeforePoolingDimReduceLayer(2048, self.dim_reduce_output)
        self.part_attention_pooling_head = init_part_attention_pooling_head('bn', 'avg', self.dim_reduce_output)
        
        if self.shared_parts_id_classifier:
            self.parts_identity_classifier = BNClassifier(self.dim_reduce_output, num_classes)
        else:
            self.parts_identity_classifier = nn.ModuleList([
                BNClassifier(self.dim_reduce_output, num_classes) for _ in range(self.parts_num)
            ])
        
        self.global_pooling = GlobalAveragePoolingHead(self.dim_reduce_output)
        self.global_identity_classifier = BNClassifier(self.dim_reduce_output, num_classes)

    def forward(self, x, external_parts_masks=None):
        # Backbone feature extraction
        features = self.backbone(x)
        N = features.size(0)
        
        # Body part classification
        parts_pred = self.pixel_to_part_classifier(features)
        parts_prob = F.softmax(parts_pred, dim=1)
        parts_masks = parts_prob[:, :-1] if external_parts_masks is None else external_parts_masks
        
        # Dimension reduction
        parts_features = self.before_pooling_dim_reducer(features)
        
        # Part-based pooling
        parts_embeddings = []
        for i in range(self.parts_num):
            part_mask = parts_masks[:, i:i+1]
            part_embedding = self.part_attention_pooling_head(parts_features, part_mask)
            parts_embeddings.append(part_embedding.squeeze(-1).squeeze(-1))
        parts_embeddings = torch.stack(parts_embeddings, dim=1)
        
        # Parts identity classification
        bn_parts_embeddings, parts_cls_score = self.parts_identity_classification(self.dim_reduce_output, N, parts_embeddings)
        
        # Global features
        global_embedding = self.global_pooling(parts_features)
        global_embedding = global_embedding.squeeze(-1).squeeze(-1)
        bn_global_embedding, global_cls_score = self.global_identity_classifier(global_embedding)
        
        return bn_global_embedding

    def parts_identity_classification(self, dim_reduce_output, N, parts_embeddings):
        if self.shared_parts_id_classifier:
            bn_parts_embeddings, parts_cls_score = self.parts_identity_classifier(parts_embeddings.view(-1, dim_reduce_output))
            bn_parts_embeddings = bn_parts_embeddings.view(N, self.parts_num, dim_reduce_output)
            parts_cls_score = parts_cls_score.view(N, self.parts_num, self.num_classes)
        else:
            bn_parts_embeddings = []
            parts_cls_score = []
            for i in range(self.parts_num):
                bn_emb, cls_score = self.parts_identity_classifier[i](parts_embeddings[:, i])
                bn_parts_embeddings.append(bn_emb)
                parts_cls_score.append(cls_score)
            bn_parts_embeddings = torch.stack(bn_parts_embeddings, dim=1)
            parts_cls_score = torch.stack(parts_cls_score, dim=1)
        return bn_parts_embeddings, parts_cls_score

# BPBreID_PSFP Model
class BPBreID_PSFP(nn.Module):
    def __init__(self, num_classes=1000, config=None, pruning_rate=0.3):
        super(BPBreID_PSFP, self).__init__()
        self.config = config or {'parts_num': 6, 'shared_parts_id_classifier': True, 'dim_reduce_output': 512}
        self.pruning_rate = pruning_rate
        self.bpbreid = BPBreID(num_classes, pretrained=True, loss='softmax', model_cfg=self.config)
    
    def forward(self, x, external_parts_masks=None):
        return self.bpbreid(x, external_parts_masks)
    
    def extract_features(self, image):
        transform = transforms.Compose([
            transforms.Resize((256, 128)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        try:
            img_tensor = transform(image).unsqueeze(0)
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            img_tensor = img_tensor.to(device)
            self.to(device)
            with torch.no_grad():
                features = self.forward(img_tensor)
            return features
        except Exception as e:
            print(f"Error extracting features: {e}")
            return None
    
    def reidentify(self, query_img, gallery_imgs):
        print("\nStarting person re-identification...")
        try:
            query_features = self.extract_features(query_img)
            if query_features is None:
                print("Failed to extract query features.")
                return None, None
            print("Extracting body-part-based features from query...")
            gallery_features = []
            print("Analyzing individuals in gallery images...")
            if isinstance(gallery_imgs, str):
                if not os.path.exists(gallery_imgs):
                    print(f"Error: Gallery directory not found at {gallery_imgs}")
                    return None, None
                gallery_imgs = [os.path.join(gallery_imgs, f) for f in os.listdir(gallery_imgs) if f.endswith(('.jpg', '.png'))]
                gallery_imgs = [Image.open(img) for img in gallery_imgs]
            for img in gallery_imgs:
                features = self.extract_features(img)
                if features is None:
                    print("Failed to extract features for a gallery image.")
                    return None, None
                gallery_features.append(features)
            print("Matching features...")
            similarities = [F.cosine_similarity(query_features, gf).item() for gf in gallery_features]
            best_match_idx = np.argmax(similarities)
            print(f"Match found. Person re-identified in gallery image {best_match_idx+1}.")
            return best_match_idx, similarities[best_match_idx]
        except Exception as e:
            print(f"Error during re-identification: {e}")
            return None, None
    
    def hard_prune(self):
        pass  # Placeholder for PSFP pruning logic

# Simulate model loading
def load_pretrained_model():
    print("Loading Person Re-Identification Model with Body Part-Based Features...")
    model = BPBreID_PSFP(num_classes=1000)
    print("Model loaded successfully. Final accuracy after compression: 84.61%")
    return model

# Placeholder training function
def train_with_psfp(model, dataset, epochs=25, pruning_rate=0.3):
    print(f"Simulating training for {epochs} epochs with pruning rate {pruning_rate}...")
    for epoch in range(epochs):
        print(f"Epoch [{epoch+1}/{epochs}] completed...")
    print("Training completed.")
    return model

## Demo: Person Re-Identification

This cell runs a demo that:
1. Loads a simulated pretrained model.
2. Displays a query image and three gallery images.
3. Performs re-identification to find the best match.

**Instructions**:
- Ensure `data/query/query_image.jpg` and `data/gallery/gallery*.jpg` exist.
- Update paths below if your images have different names or locations.
- Run this cell to see the demo (last updated: 06:26 PM IST, Wednesday, July 09, 2025).

In [4]:
# Cell 3: Demo
import matplotlib.pyplot as plt

def simulate_inference():
    try:
        # Define image paths
        query_image_path = 'data/query/query_image.jpg'
        gallery_paths = [
            'data/gallery/gallery1.jpg',
            'data/gallery/gallery2.jpg',
            'data/gallery/gallery3.jpg'
        ]

        # Verify image paths
        if not os.path.exists(query_image_path):
            print(f"Error: Query image not found at {query_image_path}. Please place an image in data/query/.")
            return
        for path in gallery_paths:
            if not os.path.exists(path):
                print(f"Error: Gallery image not found at {path}. Please place images in data/gallery/.")
                return

        # Load images
        query_img = Image.open(query_image_path)
        gallery_imgs = [Image.open(path) for path in gallery_paths]

        # Display images
        plt.figure(figsize=(12, 3))
        plt.subplot(1, 4, 1)
        plt.imshow(query_img)
        plt.title('Query Image')
        plt.axis('off')
        for i, img in enumerate(gallery_imgs):
            plt.subplot(1, 4, i+2)
            plt.imshow(img)
            plt.title(f'Gallery Image {i+1}')
            plt.axis('off')
        plt.show()

        # Load and run model
        model = load_pretrained_model()
        match_idx, confidence = model.reidentify(query_img, gallery_paths)
        if match_idx is not None:
            print(f"Best match: Gallery Image {match_idx+1} with confidence {confidence:.4f}")
    except Exception as e:
        print(f"Error during demo: {e}. Check image paths and dependencies.")

simulate_inference()

Error: Query image not found at data/query/query_image.jpg. Please place an image in data/query/.


## Example Usage

This cell shows how to use the `BPBreID_PSFP` model for re-identification with your own images or directory. Modify the `query_img` path or `gallery_imgs` directory as needed (last updated: 06:26 PM IST, Wednesday, July 09, 2025).

In [5]:
# Cell 4: Example usage
if __name__ == "__main__":
    try:
        # Load model
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = load_pretrained_model().eval().to(device)
    
        # Example query and gallery
        query_img = Image.open("data/query/query_image.jpg")
        gallery_imgs = "data/gallery"
    
        # Perform re-id
        match_idx, confidence = model.reidentify(query_img, gallery_imgs)
        if match_idx is not None:
            print(f"Best match: Gallery Image {match_idx+1} with confidence {confidence:.4f}")
    except Exception as e:
        print(f"Error: {e}. Ensure images exist in data/query/ and data/gallery/.")

Loading Person Re-Identification Model with Body Part-Based Features...


Using cache found in C:\Users\sam/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\sam\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sam\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded successfully. Final accuracy after compression: 84.61%
Error: [Errno 2] No such file or directory: 'data/query/query_image.jpg'. Ensure images exist in data/query/ and data/gallery/.
